In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [2]:
tf.__version__

'2.6.0'

Data Preprocessing

In [6]:
dataset=pd.read_csv('/content/Churn_Modelling.csv')
X = dataset.iloc[:,3 :-1].values
y = dataset.iloc[:, -1].values

In [7]:
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [8]:
print(y)

[1 0 1 ... 1 1 0]


Label Encoding the Gender column

In [10]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
X[:,2]=le.fit_transform(X[:,2])
print(X)


[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


One hot encoding the 'geography' column

In [12]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct=ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[1])],remainder='passthrough')
X=np.array(ct.fit_transform(X))

In [13]:
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


Splitting the dataset into training and testing dataset

In [14]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=1)

Feature Scaling

In [17]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)

Building the ANN

Initialize the ANN

In [18]:
ann=tf.keras.models.Sequential()

Adding the input layer and the first hidden layer

In [19]:
ann.add(tf.keras.layers.Dense(units=6,activation='relu'))

Adding the second hidden layer

In [20]:
ann.add(tf.keras.layers.Dense(units=6,activation='relu'))

Adding the output layer

In [21]:
ann.add(tf.keras.layers.Dense(units=1,activation='sigmoid'))

Training the ANN 

Compiling the ANN

In [22]:
ann.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

Training the ANN on the training set

In [23]:
ann.fit(X_train,y_train,batch_size=32,epochs=100)

Epoch 1/100
250/250 [==============================] - 1s 1ms/step - loss: 0.5248 - accuracy: 0.7972
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4657 - accuracy: 0.7972
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4457 - accuracy: 0.7972
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4345 - accuracy: 0.8016
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4246 - accuracy: 0.8069
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4140 - accuracy: 0.8098
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4020 - accuracy: 0.8136
Epoch 8/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3912 - accuracy: 0.8238
Epoch 9/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3829 - accuracy: 0.8356
Epoch 10/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3768 - accura

Making the predictions and evaluating the model

In [31]:
print(ann.predict(sc.transform([[1,0,0,600,1,40,3,30000,2,1,1,50000 ]]))>0.5)

[[False]]


Predicting the test set results

In [32]:
y_pred = ann.predict(X_test)
y_pred=(y_pred>0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[0 0]
 [0 0]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


Confusion matrix

In [33]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1541   44]
 [ 218  197]]


0.869